## Import Data

In [24]:
import sys
import os
sys.path.append(os.path.abspath('../app'))

from data import Database

In [25]:
db = Database("monsters")
df = db.dataframe()
df.head()

,Name,Type,Level,Rarity,Damage,Health,Energy,Sanity,Timestamp
0,Faerie Dragon,Dragon,4,Rank 3,4d8+3,31.53,33.88,29.55,2024-03-28 14:22:54
1,Hell Hound,Demonic,1,Rank 3,1d8,5.44,10.65,6.87,2024-03-28 14:22:54
2,Kobold Knight,Devilkin,9,Rank 4,9d10,90.11,88.39,90.53,2024-03-28 14:22:54
3,Flame Elemental,Elemental,8,Rank 1,8d4,32.98,32.94,31.27,2024-03-28 14:22:54
4,Wyvern,Dragon,6,Rank 4,6d10,63.24,62.92,61.66,2024-03-28 14:22:54


## Split Data

In [26]:
from sklearn.model_selection import train_test_split

In [27]:
X = df[['Health', 'Energy', 'Sanity']]
y = df['Rarity']

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Establish Baseline

In [31]:
from sklearn.dummy import DummyClassifier

In [32]:
dummy_model = DummyClassifier(strategy='most_frequent', random_state=42)
dummy_model.fit(X_train, y_train)
print("Training Accuracy: {:.2f}".format(dummy_model.score(X_train, y_train)))
print("Test Accuracy: {:.2f}".format(dummy_model.score(X_test, y_test)))

Training Accuracy: 0.29
Test Accuracy: 0.32


## Model 1: Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [33]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
print("Training Accuracy: {:.2f}".format(rf_model.score(X_train, y_train)))
print("Test Accuracy: {:.2f}".format(rf_model.score(X_test, y_test)))

Training Accuracy: 1.00
Test Accuracy: 0.52


## Model 2: Gradient Boosting

In [34]:
from sklearn.ensemble import GradientBoostingClassifier

In [35]:
gb_model = GradientBoostingClassifier(random_state=42)
gb_model.fit(X_train, y_train)
print("Training Accuracy: {:.2f}".format(gb_model.score(X_train, y_train)))
print("Test Accuracy: {:.2f}".format(gb_model.score(X_test, y_test)))

Training Accuracy: 0.92
Test Accuracy: 0.48


## Model 3: Logistic Regression

In [36]:
from sklearn.linear_model import LogisticRegression

In [37]:
lr_model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000, random_state=42)
lr_model.fit(X_train, y_train)
print("Training Accuracy: {:.2f}".format(lr_model.score(X_train, y_train)))
print("Test Accuracy: {:.2f}".format(lr_model.score(X_test, y_test)))

Training Accuracy: 0.45
Test Accuracy: 0.46


/home/anthony/Clones/pyenv/versions/Bandersnatch/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
